# 📊 Analysis: Column Mapping Justification

This notebook explores the statistical and structural relationships between the columns to justify the inferred mapping.

# Finding volume
Since volume is the number of traded stocks, it must be an integer value and greater than all other values in the row

In [2]:
import pandas as pd

mapping = pd.read_json("mapping.json")
df = pd.read_csv("02_sample_data_with_fabricated_columns.csv")

In [3]:
is_integer = (df["neutronCount"] % 1 == 0).all()
other_cols = df.drop(columns=["neutronCount"])
is_larger = (df["neutronCount"] > other_cols.max(axis=1)).all()
if (is_integer and is_larger):
    print("neutronCount is volume")

neutronCount is volume


# Finding high and low
We see that 2 columns - `gamma` and `omega` are greater than and less than (resprectively) every other remanining column, except `pulse`.

In [4]:
dx = df["deltaX"]
gamma = df["gamma"]
omega = df["omega"]
flux = df["flux"]
pulse = df["pulse"]

for a, b, c, d in zip(dx, gamma, omega, flux):
    if b < a or b < c or b < d:
        print("Violation!!")
    
print("gamma is high")

for a, b, c, d in zip(dx, gamma, omega, flux):
    if a < c or b < c or d < c:
        print("Violation!!")
    
print("omega is low")


gamma is high
omega is low


Since low <= open <= high and low <= close <= high, `pulse` must be price, since it is the only column to not follow this trend

Now we are tasked with mapping `deltaX` and `flux` to open and close. To do this we use the following methods

# Using regression model
* We can use the data of the past days to predict the present movement of the stock. Thus, we fit a regression model to predict the change in opening price given the momentum (close - open) of the past 4 days.
* The hypothesis is that positive momentum should cause growth and negative momentum should cause decline. Thus, if we use the correct mappings, we should get positive values of the coefficients.



In [5]:
X = []
y = []

for i in range (5, len(dx)):
    nx = []
    for j in range (4):
        nx.append(-(dx[i-5+j] - flux[i-5+j]))


    X.append(nx)
    y.append(dx[i] - dx[i-1])

import statsmodels.api as sm
import numpy as np

X = np.array(X)
y = np.array(y)

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()


print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.583
Date:                Mon, 14 Jul 2025   Prob (F-statistic):              0.176
Time:                        01:17:13   Log-Likelihood:            -4.3248e+05
No. Observations:              499995   AIC:                         8.650e+05
Df Residuals:                  499990   BIC:                         8.650e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0006      0.001     -0.721      0.4

# Observations
* We see that the coefficients are positive, albeit very small. Also, recent days get a greater coefficient than older ones (which is expected).
* To get confidence values, we check the number of predictions for which the parity (+ve/-ve) matches the actual parity. This number is very close to 0.5 (random chance), but shows a small bias towards `deltaX` ~ open and `flux` ~ close

In [6]:
y_pred = model.predict(X)
conf = sum(a * b > 0 for a, b in zip(y, y_pred)) / len(y_pred)
print(conf)

0.5017030170301703


## A Self-Defined Custom Sliding Window DP Algorithm

### Working of the DP Algorithm

1. **Find candidates**
   - Figures out which columns could be Open or Close:
`oc_candidates = all columns MINUS known High, Low, Price and Volume.`

2. **Define all possible (Open, Close) pairs**

3. **Run a dynamic programming loop**

   - For each time step (row in the data):
       - For each possible (Open, Close) pair:
           - Compute the cost of transitioning from the previous pair to this one.
           - Cost = difference between today’s Open and yesterday’s Close, plus an extra penalty if you switch mappings.
   - Store the minimum cost path.

4. **Backtrack**
   - After the forward pass, backtrace to get the optimal sequence of states (mappings).

5. **Find the most frequent mapping**
   - The most common (Open, Close) pair along the optimal path is assumed to be the real mapping.

In [10]:
import pandas as pd
import numpy as np
from itertools import permutations

DATA_FILE_PATH = "02_sample_data_with_fabricated_columns.csv"

KNOWN_HIGH = 'gamma'
KNOWN_LOW = 'omega'
KNOWN_VOLUME = 'neutronCount'
KNOWN_PRICE = 'pulse'

def solve_open_close_with_dp(file_path, high_col, low_col, volume_col, price_column, switching_penalty=0.1):
    print("Dynamic Programming Solver for Open/Close")
    
    df = pd.read_csv(file_path)
    if 'Unnamed: 0' in df.columns:
        df = df.drop(columns=['Unnamed: 0'])

    known_cols = [high_col, low_col, volume_col, price_column]
    oc_candidates = [c for c in df.columns if c not in known_cols]
    
    states = list(permutations(oc_candidates, 2))
    num_states = len(states)
    num_timesteps = len(df)
    
    print(f"Candidates for Open/Close: {oc_candidates}")
    print(f"Number of possible states (mappings): {num_states}")
    for i, state in enumerate(states):
        print(f"  State {i}: Open={state[0]}, Close={state[1]}")

    # cost_matrix[t, k]: min cost to be in state k at time t
    # backpointer_matrix[t, k]: which state at t-1 led to the min cost for state k at t
    cost_matrix = np.full((num_timesteps, num_states), np.inf)
    backpointer_matrix = np.zeros((num_timesteps, num_states), dtype=int)
    
    # At t=0, the cost to be in any state is 0 (no previous day to compare to)
    cost_matrix[0, :] = 0

    print("\nRunning forward pass of the Viterbi algorithm...")
    for t in range(1, num_timesteps):
        if t % 10000 == 0:
            print(f"Processing timestep {t}/{num_timesteps}")
            
        for j, current_state in enumerate(states):
            open_j = current_state[0]
            
            min_cost = np.inf
            best_prev_state_idx = -1
            
            for i, prev_state in enumerate(states):
                close_i = prev_state[1]
                
                # Cost = Previous Path Cost + Transition Cost + Switching Penalty
                transition_cost = abs(df[open_j].iloc[t] - df[close_i].iloc[t-1])
                penalty = switching_penalty if i != j else 0
                
                total_cost = cost_matrix[t-1, i] + transition_cost + penalty
                
                if total_cost < min_cost:
                    min_cost = total_cost
                    best_prev_state_idx = i
            
            cost_matrix[t, j] = min_cost
            backpointer_matrix[t, j] = best_prev_state_idx
            
    optimal_path = np.zeros(num_timesteps, dtype=int)
    
    optimal_path[-1] = np.argmin(cost_matrix[-1, :])
    
    for t in range(num_timesteps - 2, -1, -1):
        optimal_path[t] = backpointer_matrix[t + 1, optimal_path[t + 1]]

    print("\n--- Results ---")
    path_in_words = [f"O={states[i][0]}, C={states[i][1]}" for i in optimal_path]
    results_df = pd.DataFrame({'Optimal Mapping': path_in_words})
    
    most_likely_mapping_counts = results_df['Optimal Mapping'].value_counts()
    
    print("The most frequent mapping found across the entire dataset was:")
    print(most_likely_mapping_counts.head(1))
    
    return most_likely_mapping_counts

if __name__ == "__main__":
    most_likely_mapping = solve_open_close_with_dp(DATA_FILE_PATH, high_col=KNOWN_HIGH, low_col=KNOWN_LOW, volume_col=KNOWN_VOLUME, price_column=KNOWN_PRICE,switching_penalty=0.1)

Dynamic Programming Solver for Open/Close
Candidates for Open/Close: ['deltaX', 'flux']
Number of possible states (mappings): 2
  State 0: Open=deltaX, Close=flux
  State 1: Open=flux, Close=deltaX

Running forward pass of the Viterbi algorithm...
Processing timestep 10000/500000
Processing timestep 20000/500000
Processing timestep 30000/500000
Processing timestep 40000/500000
Processing timestep 50000/500000
Processing timestep 60000/500000
Processing timestep 70000/500000
Processing timestep 80000/500000
Processing timestep 90000/500000
Processing timestep 100000/500000
Processing timestep 110000/500000
Processing timestep 120000/500000
Processing timestep 130000/500000
Processing timestep 140000/500000
Processing timestep 150000/500000
Processing timestep 160000/500000
Processing timestep 170000/500000
Processing timestep 180000/500000
Processing timestep 190000/500000
Processing timestep 200000/500000
Processing timestep 210000/500000
Processing timestep 220000/500000
Processing ti

The above also shows that `deltaX` is open and `flux` is close, but that is true for only 266973 / 500000 samples, which is roughly 0.53 of the total. This is again, very close to random chance (0.5), but shows a small bias towards our previous finding (using the regression model). Thus, 

`deltaX`: open  
`flux`: close